Proyecto 1 BI
Tema: Datos Peliculas

# Librerias

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split


# Set Up Inicial

# Cargar Info

In [2]:
tweets_df = pd.read_csv('/Users/pipe/Desktop/GitHub/BI/Etapa 1/Archivos/MovieReviews.csv', sep = ',')

# Filtrando Tuits No Etiquetados

In [3]:
tweets_labeled_df = tweets_df.loc[tweets_df['sentimiento'].notnull()]

In [4]:
tweets_df['sentimiento'] = tweets_df['sentimiento'].map({'positivo': 1, 'negativo': 0})

In [5]:
X_train, X_test, y_train, y_test = train_test_split(tweets_df['review_es'], tweets_df['sentimiento'], test_size = 0.2, stratify = tweets_df['sentimiento'], random_state = 1)

# Pipeline Creation

In [8]:
from joblib import dump, load
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import tokenizer

In [11]:
# Descargando las stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

pipeline = Pipeline(
    [
    ('vectorizer', CountVectorizer(tokenizer=tokenizer.tokenizer, stop_words=stop_words, lowercase=True)),
    ('classifier', RandomForestClassifier(n_estimators=200, min_samples_split=5, max_depth=5, criterion='entropy'))
    ]
)

[nltk_data] Downloading package stopwords to /Users/pipe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:

pipe = pipeline.fit(X_train, y_train)



In [13]:
dump(pipe, 'modelo.joblib')

['modelo.joblib']

Ahora probemos que el joblib quedo bien

In [17]:
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# load the trained joblib model
model = joblib.load('assets/modelo.joblib')

# make predictions on the test dataset
y_pred = model.predict(X_test)

# evaluate the performance of the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Accuracy: {:.2f}'.format(accuracy))
print('Precision: {:.2f}'.format(precision))
print('Recall: {:.2f}'.format(recall))
print('F1-Score: {:.2f}'.format(f1))


Accuracy: 0.82
Precision: 0.80
Recall: 0.87
F1-Score: 0.83


In [19]:
import joblib

# load the trained joblib model
model2 = joblib.load('assets/modelo.joblib')

tweets_df2 = pd.read_csv('/Users/pipe/Downloads/PeliculasRevisiones/MovieReviewsPruebas.csv', sep = ',')
print(tweets_df2.columns)
data2 = tweets_df2['review_es']

# make predictions on the test dataset
pred = model2.predict(data2)


Index(['Unnamed: 0', 'review_es'], dtype='object')


In [20]:
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1])

In [30]:
message = "Creo que hace dos años desde que he visto la película y hasta el día de hoy, es la peor película que he visto.Lo único que pensé después de ver esta película fue que fue hecha para algún motivo fiscal.Así que después de todo este tiempo, finalmente derramé mi intestino;) Y ahora IMDB dice que tengo que llenar 10 líneas con comentarios: Lo sentimos, debe proporcionar al menos 10 líneas en su comentario. Por favor regrese a la ventana Editar (o use la parte posteriorOpción Si esta no es una ventana nueva). Por favor, no hay nada que decir más ... Lo siento por un mal inglés."

message = [message]

result = model2.predict(message)

In [32]:
result[0]

0